<a href="https://colab.research.google.com/github/apujais007/api-nuggets/blob/main/price_target_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
import os

# Set the OPENAI_API_KEY environment variable
os.environ["FMP_API_KEY"] = userdata.get('FMP')

print("FMP_API_KEY environment variable has been set.")

FMP_API_KEY environment variable has been set.


In [17]:
import requests

import requests
import pandas as pd
import json
from datetime import datetime
import pytz

# Define the API endpoint and your API key
API_KEY = os.environ["FMP_API_KEY"]
url = f"https://financialmodelingprep.com/stable/price-target-latest-news?apikey={API_KEY}"


resp = requests.get(url)
data = resp.json()

rows = []
utc = pytz.utc
eastern = pytz.timezone("US/Eastern")

for entry in data:
    raw_date = entry.get("publishedDate")
    if raw_date:
        dt_utc = datetime.strptime(raw_date, "%Y-%m-%dT%H:%M:%S.000Z").replace(tzinfo=utc)
        dt_est = dt_utc.astimezone(eastern)
        formatted_date = dt_est.strftime("%I:%M %p %Z")
    else:
        formatted_date = None

    rows.append({
        "symbol": entry.get("symbol"),
        "Time":formatted_date,
        "analyst": entry.get("analystCompany", ""),
        "priceWhenPosted": entry.get("priceWhenPosted"),
        "priceTarget": entry.get("priceTarget"),
        "newsTitle": entry.get("newsTitle")
    })

df = pd.DataFrame(rows)

# Define color highlighting function
def highlight_target(row):
    if pd.notnull(row['priceTarget']) and pd.notnull(row['priceWhenPosted']):
        if row['priceTarget'] > row['priceWhenPosted']:
            return ['background-color: lightgreen'] * len(row)
        elif row['priceTarget'] < row['priceWhenPosted']:
            return ['background-color: salmon'] * len(row)
    return [''] * len(row)

styled_df = df.style.apply(highlight_target, axis=1)
styled_df

,symbol,Time,analyst,priceWhenPosted,priceTarget,newsTitle
0,NVO,04:03 PM EDT,HSBC,59.170000,70,Novo Nordisk upgraded to Buy from Hold at HSBC
1,LW,02:58 PM EDT,Stifel Nicolaus,62.010000,63,Lamb Weston price target raised to $63 from $61 at Stifel
2,VUZI,11:59 AM EDT,Craig-Hallum,3.480000,6,Vuzix price target raised to $6 from $3 at Craig-Hallum
3,ARE,11:18 AM EDT,BNP Paribas,83.460000,82,Alexandria Real Estate downgraded to Underperform from Neutral at BNP Paribas Exane
4,UNFI,10:36 AM EDT,Roth Capital,39.185200,35,United Natural Foods price target raised to $35 from $25 at Roth Capital
5,ENTA,10:21 AM EDT,Jefferies,10.884900,20,Enanta upgraded to Buy from Hold at Jefferies
6,WTW,09:46 AM EDT,Evercore ISI,343.950000,387,WTW price target raised to $387 from $373 at Evercore ISI
7,AON,09:46 AM EDT,Evercore ISI,354.640000,427,Aon plc price target raised to $427 from $411 at Evercore ISI
8,FIHL,09:46 AM EDT,Evercore ISI,18.290000,19,Fidelis Insurance price target raised to $19 from $18 at Evercore ISI
9,EG,09:46 AM EDT,Evercore ISI,353.130000,384,Everest Group price target raised to $384 from $380 at Evercore ISI
